In [1]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from numba import jit
import time
from PIL import Image
import matplotlib.animation as animation
import copy
from IPython.display import HTML
%matplotlib inline

In [2]:
os.chdir('D:\\Radar Projects\\lizhi\\for LiZhi\\WindField')

curr_img = np.matrix(Image.open('test_file.tif'), dtype=np.float32)

In [3]:
@jit(nopython=True)
def grid2grid(x,y,x_dis, y_dis):
    return (int(x+x_dis/100.), int(y+y_dis/100.)) #distance divided by 100m per pixel

In [4]:
@jit(nopython=True)
def fast_iter(curr_img, x,y,D_x,D_y):
    pred_img= np.zeros((1000,1000), dtype=np.float32)
    for (m,n) in np.nditer((x,y)):
        trans_i, trans_j=grid2grid(m,n, D_y[m,n], D_x[m,n])
        if 0<=trans_i<=999 and 0<=trans_j<=999:
            pred_img[trans_i, trans_j] = curr_img[m,n]
    return pred_img

In [5]:
def find_gap(mat, axis):
    mat= mat//100
    if axis==1:
        diff_mat=mat[:,1:]-mat[:,0:-1]
        x,y=np.where(diff_mat!=0)
        y=y[y<=998]
        return x,y+1
    elif axis==0:
        diff_mat=mat[1:,:]-mat[0:-1,:]
        x,y=np.where(diff_mat!=0)
        x=x[x<=998]
        return x+1,y

In [6]:
# def find_gap(mat, axis):
#     mat= mat//100
#     if axis==1:
#         diff_mat=mat[:,1:]-mat[:,0:-1]
#     elif axis==0:
#         diff_mat=mat[1:,:]-mat[0:-1,:]
#     x,y=np.where(diff_mat!=0)
#     return x,y

In [7]:
# @jit(nopython=True)
def X_update(next_img, curr_img, steps_x, gap_y_0, gap_x_0):
    X_forward= gap_y_0+steps_x[gap_x_0, gap_y_0]
    X_forward_new= X_forward[(X_forward<=999) & (X_forward>=0)]
    gap_y_0= gap_y_0[(X_forward<=999) & (X_forward>=0)]
    gap_x_0= gap_x_0[(X_forward<=999) & (X_forward>=0)]
    for i in range(len(gap_y_0)):
        next_img[gap_x_0[i], gap_y_0[i]:X_forward_new[i]]= curr_img[gap_x_0[i], gap_y_0[i]:X_forward_new[i]].mean()
    return next_img

# @jit(nopython=True)
def Y_update(next_img, curr_img, steps_y, gap_y_1, gap_x_1):
    Y_forward= gap_x_1+steps_y[gap_x_1,gap_y_1]
    Y_forward_new= Y_forward[(Y_forward<=999) & (Y_forward>=0)]
    gap_x_1= gap_x_1[(Y_forward<=999) & (Y_forward>=0)]
    gap_y_1= gap_y_1[(Y_forward<=999) & (Y_forward>=0)]
    for i in range(len(gap_y_1)):
#         print(next_img[gap_x_1[i]:Y_forward_new[i], gap_y_1[i]], curr_img[gap_x_1[i]:Y_forward_new[i], gap_y_1[i]])
        next_img[gap_x_1[i]:Y_forward_new[i], gap_y_1[i]]=curr_img[gap_x_1[i]:Y_forward_new[i], gap_y_1[i]].mean()
    return next_img

In [8]:
def execute(curr_img, V, tmax, dt):
    pred=[]
    start=time.time()
    fig=plt.figure(figsize=(10,10))
    for t in range(60,tmax,dt):
        x,y= np.where(curr_img!=0)
        D_x,D_y=V[:,:,0]*dt, V[:,:,1]*dt
        gap_x_1,gap_y_1=find_gap(D_x,1)
        gap_x_0,gap_y_0=find_gap(D_y,0)
        steps_x, steps_y =(D_x//100).astype(int), (D_y//100).astype(int)
#         print(len(gap_x_0), len(gap_y_0))
        next_img=fast_iter(curr_img, x,y,D_x,D_y)
        #------------------Put X, Y updates together--------------------
        next_img= X_update(next_img, curr_img, steps_x, gap_y_1, gap_x_1)
        next_img= Y_update(next_img, curr_img, steps_y, gap_y_0, gap_x_0)
        #---------------------------------------------------------------
        curr_img= copy.deepcopy(next_img)
        img = plt.imshow(next_img, animated=True)
        quiver = plt.quiver(np.arange(0,1000,100), np.arange(0,1000,100), V[0:1000:100,0:1000:100,0],-V[0:1000:100,0:1000:100,1], color='white')
        text = plt.text(400,-10,f'{int(t/60)} min',color='black')
        pred.append([img, text, quiver])
        print(f'{int(t/60)} min done!')
        
    ani = animation.ArtistAnimation(fig, pred, interval=500, blit=True,
                                repeat_delay=1000)
    plt.colorbar(img)
    plt.close()
    end=time.time()
    print(f'total elapsed time: {round((end-start)/60,2)} minutes')
    return ani

In [9]:
V = np.zeros((1000,1000,2))
V[:,:100,0]=2
V[:,100:200,0]=2
V[:,200:300,0]=4
V[:,300:400,0]=6
V[:,400:500,0]=8
V[:,500:600,0]=10
V[:,600:700,0]=12
V[:,700:800,0]=14
V[:,800:900,0]=16
V[:,900:1000,0]=18

V[:100,:,1]=2
V[100:200,:,1]=2
V[200:300,:,1]=4
V[300:400,:,1]=6
V[400:500,:,1]=8
V[500:600,:,1]=10
V[600:700,:,1]=12
V[700:800,:,1]=14
V[800:900,:,1]=16
V[900:,:,1]=18

In [10]:
ani =execute(curr_img, V, 1860, 60)

0 min done!
1 min done!
2 min done!
3 min done!
4 min done!
5 min done!
6 min done!
7 min done!
8 min done!
9 min done!
10 min done!
11 min done!
12 min done!
13 min done!
14 min done!
15 min done!
16 min done!
17 min done!
18 min done!
19 min done!
20 min done!
21 min done!
22 min done!
23 min done!
24 min done!
25 min done!
26 min done!
27 min done!
28 min done!
29 min done!
30 min done!
total elapsed time: 0.19 minutes


In [11]:
HTML(ani.to_jshtml())